In [ ]:
appId = '' 
appSecret = '' 
tenantId = ''

# Playing with 'Microsoft Threat Protection' API
- Related to: Microsoft 365 Defender (M365D) 
- Documentation: https://docs.microsoft.com/en-us/microsoft-365/security/defender/api-supported
- Resource URI: https://api.security.microsoft.com
- API required: Microsoft Threat Protection
- API permissions required:
  - AdvancedHunting.Read.All
  - Incident.Read.All
- How to request access to admin: $https://login.microsoftonline.com/common/oauth2/authorize?prompt=consent&client_id=<HERE-THE-CLIENT-ID>&response_type=code&sso_reload=true$

# Required libraries

In [ ]:
import pandas as pd
import json
import urllib.parse

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get the token (for security and securitycenter)

In [ ]:
def getaadtoken(tenantId, appId, appSecret):
    url = "https://login.windows.net/%s/oauth2/token" % (tenantId)
    resourceAppIdUri = 'https://api.security.microsoft.com'
    body = {
        'resource' : resourceAppIdUri,
        'client_id' : appId,
        'client_secret' : appSecret,
        'grant_type' : 'client_credentials'
    }
    data = urllib.parse.urlencode(body).encode("utf-8")
    req = urllib.request.Request(url, data)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    aadToken = jsonResponse["access_token"]
    return aadToken

In [ ]:
access_token = getaadtoken(tenantId, appId, appSecret)

<h1 align='center'>========================================================<br>=======================================================<br> ADVANCED HUNTING API</h1>

https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-advanced-hunting?view=o365-worldwide

Note: this API is the same as the of from the MDE/MDATP. Examples of queries [here!](https://github.com/microsoft/Microsoft-365-Defender-Hunting-Queries)

# Function Run (KQL) - Still permission problems!

In [ ]:
def advancedquery_run(access_token,kql_query):
    url = "https://api.security.microsoft.com/api/advancedqueries/run"
    data = json.dumps({ 'Query' : kql_query }).encode("utf-8")    
    headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'}
    req = urllib.request.Request(url, data, headers)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['Results'])

# Example of instantiating this function

In [ ]:
kql_query= "DeviceProcessEvents | limit 10"
# 'DeviceRegistryEvents | limit 10'

In [ ]:
advancedquery_run(access_token,kql_query)

<h1 align='center'>========================================================<br>=======================================================<br>INCIDENT API</h1>

List of methods in the Incident API (https://docs.microsoft.com/en-us/microsoft-365/security/mtp/api-incident?view=o365-worldwide

|Method	| Return Type |Description  |
|:- |:-|:-|
|List incidents|	Incident list|	Get a list of incidents.|
|Update incident|	Incident|	Update a specific incident.|

## METHOD List incidents

In [ ]:
def m365d_incidents_listincidents(access_token):
    url = "https://api.security.microsoft.com/api/incidents"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

In [ ]:
df_incidents = m365d_incidents_listincidents(access_token)
df_incidents